# Ingesta de datos para los ejercicios

Vamos a crear el íncide que utilizaremos en los ejercicios e ingestar los datos necesarios para poder ejecutar las consultas.

Vamos a almacenar recetas de cocina. Lo primero que vamos a ver es el formato de los datos que vamos a insertar.

In [ ]:
import pandas as pd

df = pd.read_json("../../data/elasticsearch/recipes/recipes_copia.json", lines = True)
df.head()

## Paso 1: Crear el índice

Para crear el índice ejecuta la siguiente sentencia usando la herramienta dev tools de Kibana.

```
PUT recipes
{
    "aliases": {},
    "settings" : { 
        "index" : {
            "number_of_shards" : 4,
            "number_of_replicas" : 1
        },
        "analysis": {
          "analyzer": {
            "autocomplete": {
              "tokenizer": "autocomplete",
              "filter": [
                "lowercase"
              ]
            },
            "autocomplete_search": {
              "tokenizer": "lowercase"
            }
          },
          "tokenizer": {
            "autocomplete": {
              "type": "ngram",
              "min_gram": 1,
              "max_gram": 2,
              "token_chars": [
                "letter"
              ]
            }
          }
        }
    },
    "mappings": {
      "properties": {
        "author": {
          "type": "keyword",
          "eager_global_ordinals": true,
          "fields": {
            "text": {
              "type": "text"
            }
          }
        },
        "date": {
          "type": "date",
          "format": "[MMMM yyyy]",
          "fields": {
            "keyword": {
              "type": "keyword"
            }
          }
        },
        "description": {
          "type": "text",
          "analyzer": "english"
        },
        "ingredients": {
          "type": "text",
          "analyzer": "english"
        },
        "instructions": {
          "type": "text",
          "analyzer": "english"
        },
        "picture_link": {
          "type": "keyword"
        },
        "rating": {
          "properties": {
            "bestRating": {
              "type": "float"
            },
            "prepareAgainPct": {
              "type": "float"
            },
            "ratingValue": {
              "type": "float"
            },
            "worstRating": {
              "type": "float"
            }
          }
        },
        "recipe_id": {
          "type": "text",
          "fields": {
            "keyword": {
              "type": "keyword",
              "ignore_above": 256
            }
          }
        },
        "summary": {
          "properties": {
            "active-time": {
              "type": "keyword"
            },
            "total-time": {
              "type": "keyword"
            },
            "yield": {
              "type": "keyword"
            }
          }
        },
        "title": {
          "type": "text",
          "analyzer": "english",
          "fields": {
            "keyword": {
              "type": "keyword"
            },
            "suggestion": {
              "type": "completion",
              "analyzer": "english",
              "preserve_separators": false,
              "preserve_position_increments": false,
              "max_input_length": 50
            },
            "ngram": {
            	"type": "text",
            	"analyzer": "autocomplete",
        		"search_analyzer": "autocomplete_search"
            }
          }
        },
        "url": {
          "type": "text",
          "fields": {
            "keyword": {
              "type": "keyword",
              "ignore_above": 256
            }
          }
        }
      }
    }
  }
```

## Paso 2: Levantar Logstash

Vamos a ingestar los datos usando Logsetash usando una imagen de docker para ejecutarlo:

* Para que pueda encontrar el servicio de Elasticsearch vamos a añadir el conteneror a la red de nuestro laboratorio, `--network=datahack-elk-v1_defaul`. Puedes consultar la red creada por tu compose usando el comando `docker networks ls`
* Montamos el volumen donde se encuentra nuestro fichero con el pipeline y los referenciamos a la carpeta del contenedor donde Logstash espera encontrar esa configuración, `-v /Users/rgarrote/desarrollo/datahack-elk-v1/work/data/elasticsearch/web_logs/pipeline/:/usr/share/logstash/pipeline/`.
* Montamos el volumen donde dejaremos los ficheros de log a parsear. `-v /Users/rgarrote/desarrollo/datahack-elk-v1/work/data/elasticsearch/web_logs/data/:/tmp/data/`.

In [ ]:
docker run --rm -it --network=datahack-elk-v1_defaul \
    -v /Users/rgarrote/desarrollo/datahack-elk-v1/work/data/elasticsearch/recipies/pipeline/:/usr/share/logstash/pipeline/ \
    -v /Users/rgarrote/desarrollo/datahack-elk-v1/work/data/elasticsearch/recipies/data/:/tmp/data/ \
docker.elastic.co/logstash/logstash:8.3.3

## Paso 3: Ingestar los datos.

Una vez que Logstash haya levantado y esté listo para procesar ficheros, copia el fichero que encontrarás en la ruta `work/data/elasticsearch/recipes/recipes.json` en la carpeta `workdata/elasticsearch/recipies/data`.

Por cada documento ingestado se mostrará un punto en la pantalla.


## Paso 4: Comprobar que el proceso se está realizando correctamente

1. Comprueba en Kibana que se ha ceado el íncide recipes.
2. Desde la sección de Index Management averigua cuantas recetas se han insertado en el índice recipes de Elasticsearch.